# Generamos Fotos a partir de los datos históricos
Importamos las librerias necesarias

In [1]:
import os
import pandas as pd

pd.set_option("display.max_columns", None)

_Definiciones de ficheros_
* Reservas

In [83]:

reservas_source_path = r"C:\Datos\UOC\OneDrive - Universitat Oberta de Catalunya\TFM\Datos\Historicos\Reservas" # segundo continuamos con todos los ficheros de un directorio
reservas_dtypes = {
    "Hotel" : str,
    "Cliente" : str,
    "TTOO" : str,
    "Agencia" : str,
    "Noches" : int,
    "Cantidad habitaciones" : int,
    "Nacionalidad" : str,
    "Régimen factura" : str,
    "Tipo facturación estancia" : str,
    "Estado reserva" : str,
    "Desglosada" : int,
    "Usuario venta" : str,
    "Usuario Borrado" : str,
    "Day Use" : int,
    "Grupo" : str,
    "Master del grupo" : int,
    "Canal_Origen" : str,
    "Segmento" : str,
    "Régimen uso" : str,
    "Canal_Origen Aux" : str,
    "Id Reserva" : str,
    "Id Tipo habitacion uso" : str,
    "Id Tipo habitacion factura" : str
}
reservas_parse_dates = [
    "Fecha venta", 
    "Fecha entrada", 
    "Fecha salida",
    "Fecha Borrado"
]

reservas_cols = list(reservas_dtypes.keys()) + reservas_parse_dates + ["AD", "JR", "NI", "CU"]



* Reservas Diarias

In [ ]:

reservas_diarias_source_path = r"C:\Datos\UOC\OneDrive - Universitat Oberta de Catalunya\TFM\Datos\Historicos\ReservasDiarias" # segundo continuamos con todos los ficheros de un directorio
reservas_diarias_dtypes = {
    "Hotel" : str,
    "Régimen factura" : str,
    "Estado reserva" : str,
    "Régimen uso" : str,
    "Day Use" : int,
    "Canal_Origen" : str,
    "Segmento" : str,
    "Id Reserva" : str,
    "Id Tipo habitacion uso" : str,
    "Id Tipo habitacion factura" : str,
    "Id Habitacion" : str
}

reservas_diarias_parse_dates = ["Fecha"]

reservas_diarias_cols = list(reservas_diarias_dtypes.keys()) + reservas_diarias_parse_dates + ["Ocupación", "AD", "JR", "NI", "CU"]


Creamos funciones de uso genérico a lo largo del script

In [80]:
def getListOfFiles(path:str) -> list:
    """Get the list of all files in directory tree at given path"""
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(path):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]

    return listOfFiles

to_integer = lambda i: (int(float(i)))


El proceso sigue el siguiente algoritmo:
* Cargar todas las Reservas de un hotel en un DataSet
* Empezar a generar fotos para cada día desde la primera fecha de venta que tengamos
	* Para cada día desde MinFechaVenta hasta MaxFechaVenta
		* Filtrar Para cada Fecha en Lista
			* FechaVenta <= Fecha y FechaBorrado >= Fecha y Estado En Reserva, Entrada, Salida
				* Generar un fichero para ese día con todos los datos
				
_Unificamos las Reservas_

In [78]:
files = getListOfFiles(reservas_source_path)

all_years_reservas = pd.DataFrame()
for file in files:   
    current_data = pd.read_csv(
                                file, 
                                sep="|", 
                                escapechar="~",
                                encoding="cp1252", 
                                on_bad_lines="skip", 
                                encoding_errors="ignore",
                                quoting=3,
                                usecols=reservas_cols,
                                dtype=reservas_dtypes, 
                                parse_dates=reservas_parse_dates,
                                converters={"AD":to_integer, "JR":to_integer, "NI":to_integer, "CU":to_integer}                                
                            )
    all_years_reservas = pd.concat([all_years_reservas, current_data], ignore_index=True)
   
all_years_reservas.to_csv("data/processed/all_data_historico_reservas.csv", index=False, sep="|", escapechar="~", encoding="cp1252")
all_years_reservas.to_parquet("data/processed/all_data_historico_reservas.parquet", index=False)

_Unificamos las Reservas Diarias_

In [84]:
files = getListOfFiles(reservas_diarias_source_path)

all_years_reservas_diarias = pd.DataFrame()
for file in files:
    current_data = pd.read_csv(
                                file, 
                                sep="|", 
                                escapechar="~",
                                encoding="cp1252", 
                                on_bad_lines="skip", 
                                encoding_errors="ignore",
                                quoting=3,
                                usecols=reservas_diarias_cols,
                                dtype=reservas_diarias_dtypes, 
                                parse_dates=reservas_diarias_parse_dates, 
                                converters={"Ocupación":to_integer, "AD":to_integer, "JR":to_integer, "NI":to_integer, "CU":to_integer} 
                            )
    all_years_reservas_diarias = pd.concat([all_years_reservas_diarias, current_data], ignore_index=True)
    
all_years_reservas_diarias.to_csv("data/processed/all_data_historico_reservas_diarias.csv", index=False, sep="|", escapechar="~", encoding="cp1252")
all_years_reservas_diarias.to_parquet("data/processed/all_data_historico_reservas_diarias.parquet", index=False)

_Generación de Fotos Diarias_

In [87]:
reservas = pd.read_parquet(r"data\processed\all_data_historico_reservas.parquet")
reservas_diarias = pd.read_parquet(r"data\processed\all_data_historico_reservas_diarias.parquet")

In [126]:
# reservas.info()
# Limpieza datos:
#   Eliminar Reservas Anuladas con estancia mayor a 1 año
reservas[(reservas["Noches"] > 35) & (reservas["Estado reserva"] != "Salida")][["Hotel","Estado reserva","Id Reserva", "Noches"]].head(10)



,Hotel,Estado reserva,Id Reserva,Noches
11456,02,Anulada,02-2014-4799-1,39
12996,02,Anulada,02-2014-1442-1,36
38924,02,Anulada,02-2016-8324-1,189
62464,02,Anulada,02-2011-1126R-1,2406
97869,02,Anulada,02-2021-148-1,41


In [95]:

reservas_diarias.head(10)

,Hotel,Fecha,Régimen factura,Estado reserva,Régimen uso,Ocupación,AD,JR,NI,CU,Day Use,Canal_Origen,Segmento,Id Reserva,Id Tipo habitacion uso,Id Tipo habitacion factura,Id Habitacion
0,02,2013-04-19,AI,Salida,AI,1,2,2,0,0,0,None,None,02-2013-2565-1,02-RS,02-A,02-8111
1,02,2013-04-19,AI,Salida,AI,1,2,0,1,0,0,None,None,02-2013-2596-1,02-RS,02-A,02-8117
2,02,2013-04-19,MPP,Salida,MPP,1,2,3,0,0,0,None,None,02-2013-55-1,02-SCA,02-PRIA,02-8009
3,02,2013-04-19,AL,Salida,AL,1,2,0,2,0,0,None,None,02-2013-779-1,02-SCA,02-SCA,02-7101
4,02,2013-04-19,MPP,Salida,MPP,1,2,0,0,0,0,None,None,02-2013-2200-1,02-SCA,02-AM,02-8011
5,02,2013-04-19,AL,Salida,AL,1,2,0,0,0,0,None,None,02-2013-2804-1,02-SCA,02-S,02-8105
6,02,2013-04-19,MPP,Salida,MPP,1,2,0,1,1,0,None,None,02-2013-670-1,02-SCPA,02-A,02-8005
7,02,2013-04-19,AL,Salida,AL,1,2,0,0,0,0,None,None,02-2013-1374-1,02-SCA,02-A,02-8103
8,02,2013-04-19,AL,Salida,AL,1,2,0,0,0,0,None,None,02-2013-1394-1,02-SCA,02-AM,02-8015
9,02,2013-04-19,AI,Salida,AI,1,2,0,2,0,0,None,None,02-2013-2169-1,02-RS,02-RS,02-8115
